# Bundling Analysis

In [6]:
# libraries
library(tidyverse)
library(arules)
library(arulesViz)
library(RPostgreSQL)
library(tsibble)
library(gridExtra)
library(clipr)
library(stringr)
library(config)
library(googlesheets4)

# setting wd
if ( grepl("bundles", getwd(), fixed = TRUE) ) {
    print("current working directory is fine")
} else {
    setwd("./projects/bundles")
    print("changed working directory")
}

[1] "current working directory is fine"


In [7]:
# load functions
source("bundle-functions.R")

### Get Transactions

In [8]:
# get credentials
dw <- config::get(file="./redshift_credentials.yml")

In [24]:
options(warn=-1)
# connecting to dw
con <- DBI::dbConnect(RPostgreSQL::PostgreSQL(),
                      dbname = dw$dbname,
                      host = dw$host,
                      port = dw$port,
                      user = dw$user,
                      password = dw$password)
# read query from sql file and execute
query_1 <- getSQL("sql/data_pull__purchases.sql") 
df_purchases <- dbGetQuery(con, query_1) %>% as_tibble()

query_2 <- getSQL("sql/data_pull__purchases_same_size_by_size.sql") 
df_purchases_same_size_by_size <- dbGetQuery(con, query_2) %>% as_tibble()

query_3 <- getSQL("sql/data_pull__purchases_same_size_overall.sql") 
df_purchases_same_size_overall <- dbGetQuery(con, query_3) %>% as_tibble()

query_4 <- getSQL("sql/data_pull__top-selling-designs.sql") 
df_top_designs <- dbGetQuery(con, query_4) %>% as_tibble()

options(warn=0)

In [25]:
options(warn=-1)
purchase_transactions <- makeTransaction(df_purchases)
samesize_bysize_purchase_transactions <- makeTransaction(df_purchases_same_size_by_size)
samesize_overall_purchase_transactions <- makeTransaction(df_purchases_same_size_overall)
options(warn=0)

In [26]:
summary(samesize_overall_purchase_transactions)

transactions as itemMatrix in sparse format with
 156898 rows (elements/itemsets/transactions) and
 1498 columns (items) and a density of 0.001482259 

most frequent items:
         kamran coral rug hendesi heriz abalone rug  damali black & white rug 
                     7452                      6478                      5791 
         kamran hazel rug      verena dark wood rug                   (Other) 
                     5464                      4714                    318481 

element (itemset/transaction) length distribution:
sizes
     2      3      4      5      6      7      8      9     10     11     12 
131559  19326   4198   1139    358    160     72     35     17     14      6 
    13     14     15     17     18     19     26     36 
     5      1      2      2      1      1      1      1 

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
   2.00    2.00    2.00    2.22    2.00   36.00 

includes extended item information - examples:
                           labels
1 

In [27]:
select_rug_rules <- apriori(samesize_overall_purchase_transactions, parameter = list(supp=0.00001, conf=0.01),appearance = list(lhs="domo sage rug",default="rhs"))

Apriori

Parameter specification:
 confidence minval smax arem  aval originalSupport maxtime support minlen
       0.01    0.1    1 none FALSE            TRUE       5   1e-05      1
 maxlen target  ext
     10  rules TRUE

Algorithmic control:
 filter tree heap memopt load sort verbose
    0.1 TRUE TRUE  FALSE TRUE    2    TRUE

Absolute minimum support count: 1 

set item appearances ...[1 item(s)] done [0.00s].
set transactions ...[1498 item(s), 156898 transaction(s)] done [0.08s].
sorting and recoding items ... [1478 item(s)] done [0.00s].
creating transaction tree ... done [0.05s].
checking subsets of size 1 2 done [0.00s].
writing ... [66 rule(s)] done [0.00s].
creating S4 object  ... done [0.02s].


In [29]:
inspect(subset(select_rug_rules,lift > 1.2))

     lhs                rhs                                        support confidence    coverage      lift count
[1]  {domo sage rug} => {moden blue rug}                      3.824141e-05 0.02419355 0.001580645 23.145850     6
[2]  {domo sage rug} => {zareen scarlet red rug - cover only} 2.549427e-05 0.01612903 0.001580645 10.861000     4
[3]  {domo sage rug} => {mariposa sage rug}                   5.098854e-05 0.03225806 0.001580645 15.198876     8
[4]  {domo sage rug} => {solid bordered natural rug}          1.912070e-05 0.01209677 0.001580645  5.804158     3
[5]  {domo sage rug} => {kilim batik pink rug}                1.912070e-05 0.01209677 0.001580645  5.272110     3
[6]  {domo sage rug} => {cynthia rowley lamella jade rug}     1.912070e-05 0.01209677 0.001580645  5.088364     3
[7]  {domo sage rug} => {kenza charcoal rug}                  1.912070e-05 0.01209677 0.001580645  4.303763     3
[8]  {domo sage rug} => {kavi diamond silver rug}             2.549427e-05 0.01612903 0.

In [28]:
rules_ss_overall_purchases <- apriori(samesize_overall_purchase_transactions, parameter = list(supp=0.00001, conf=0.2,maxlen=10))

Apriori

Parameter specification:
 confidence minval smax arem  aval originalSupport maxtime support minlen
        0.2    0.1    1 none FALSE            TRUE       5   1e-05      1
 maxlen target  ext
     10  rules TRUE

Algorithmic control:
 filter tree heap memopt load sort verbose
    0.1 TRUE TRUE  FALSE TRUE    2    TRUE

Absolute minimum support count: 0 

set item appearances ...[0 item(s)] done [0.00s].
set transactions ...[14236 item(s), 20932 transaction(s)] done [0.01s].
sorting and recoding items ... [14236 item(s)] done [0.00s].
creating transaction tree ... done [0.01s].
checking subsets of size 1 done [0.00s].
writing ... [0 rule(s)] done [0.00s].
creating S4 object  ... done [0.00s].


In [625]:
rules_purchases <- apriori(purchase_transactions, parameter = list(supp=0.00001, conf=0.2,maxlen=10))
rules_ss_bysize_purchases <- apriori(samesize_bysize_purchase_transactions, parameter = list(supp=0.00001, conf=0.2,maxlen=10))
rules_ss_overall_purchases <- apriori(samesize_overall_purchase_transactions, parameter = list(supp=0.00001, conf=0.2,maxlen=10))
rules_pageviews <- apriori(pageview_transactions, parameter = list(supp=0.001, conf=0.2,maxlen=10))
rules_favorites <- apriori(favorite_transactions, parameter = list(supp=0.001, conf=0.2,maxlen=10))

Apriori

Parameter specification:
 confidence minval smax arem  aval originalSupport maxtime support minlen
        0.2    0.1    1 none FALSE            TRUE       5   1e-05      1
 maxlen target  ext
     10  rules TRUE

Algorithmic control:
 filter tree heap memopt load sort verbose
    0.1 TRUE TRUE  FALSE TRUE    2    TRUE

Absolute minimum support count: 0 

set item appearances ...[0 item(s)] done [0.00s].
set transactions ...[7796 item(s), 63652 transaction(s)] done [0.05s].
sorting and recoding items ... [7796 item(s)] done [0.00s].
creating transaction tree ... done [0.02s].
checking subsets of size 1 2 3 4 5 6 7 8 9 10

Warning message in apriori(purchase_transactions, parameter = list(supp = 1e-05, :
“Mining stopped (maxlen reached). Only patterns up to a length of 10 returned!”


 done [2.91s].
writing ... [21891638 rule(s)] done [4.10s].
creating S4 object  ... done [7.66s].
Apriori

Parameter specification:
 confidence minval smax arem  aval originalSupport maxtime support minlen
        0.2    0.1    1 none FALSE            TRUE       5   1e-05      1
 maxlen target  ext
     10  rules TRUE

Algorithmic control:
 filter tree heap memopt load sort verbose
    0.1 TRUE TRUE  FALSE TRUE    2    TRUE

Absolute minimum support count: 0 

set item appearances ...[0 item(s)] done [0.00s].
set transactions ...[5821 item(s), 18466 transaction(s)] done [0.01s].
sorting and recoding items ... [5821 item(s)] done [0.00s].
creating transaction tree ... done [0.00s].
checking subsets of size 1 2 3 4 5 6 7 8 9 10

Warning message in apriori(samesize_bysize_purchase_transactions, parameter = list(supp = 1e-05, :
“Mining stopped (maxlen reached). Only patterns up to a length of 10 returned!”


 done [0.18s].
writing ... [52723 rule(s)] done [0.08s].
creating S4 object  ... done [0.02s].
Apriori

Parameter specification:
 confidence minval smax arem  aval originalSupport maxtime support minlen
        0.2    0.1    1 none FALSE            TRUE       5   1e-05      1
 maxlen target  ext
     10  rules TRUE

Algorithmic control:
 filter tree heap memopt load sort verbose
    0.1 TRUE TRUE  FALSE TRUE    2    TRUE

Absolute minimum support count: 0 

set item appearances ...[0 item(s)] done [0.00s].
set transactions ...[1365 item(s), 18466 transaction(s)] done [0.01s].
sorting and recoding items ... [1365 item(s)] done [0.00s].
creating transaction tree ... done [0.00s].
checking subsets of size 1 2 3 4 5 6 7 8 9 10

Warning message in apriori(samesize_overall_purchase_transactions, parameter = list(supp = 1e-05, :
“Mining stopped (maxlen reached). Only patterns up to a length of 10 returned!”


 done [0.03s].
writing ... [42337 rule(s)] done [0.01s].
creating S4 object  ... done [0.01s].
Apriori

Parameter specification:
 confidence minval smax arem  aval originalSupport maxtime support minlen
        0.2    0.1    1 none FALSE            TRUE       5   0.001      1
 maxlen target  ext
     10  rules TRUE

Algorithmic control:
 filter tree heap memopt load sort verbose
    0.1 TRUE TRUE  FALSE TRUE    2    TRUE

Absolute minimum support count: 71 

set item appearances ...[0 item(s)] done [0.00s].
set transactions ...[817 item(s), 71873 transaction(s)] done [0.04s].
sorting and recoding items ... [502 item(s)] done [0.00s].
creating transaction tree ... done [0.02s].
checking subsets of size 1 2 3 4 5 6 done [0.02s].
writing ... [2761 rule(s)] done [0.00s].
creating S4 object  ... done [0.01s].
Apriori

Parameter specification:
 confidence minval smax arem  aval originalSupport maxtime support minlen
        0.2    0.1    1 none FALSE            TRUE       5   0.001      1
 m

#### Top rules sorted by count

In [626]:
df_rules_pu <- DATAFRAME(rules_purchases,separate=TRUE) %>% as_tibble()
df_rules_ss_bysize <- DATAFRAME(rules_ss_bysize_purchases,separate=TRUE) %>% as_tibble() 
df_rules_ss_overall <- DATAFRAME(rules_ss_overall_purchases,separate=TRUE) %>% as_tibble()
df_rules_pv <- DATAFRAME(rules_pageviews,separate=TRUE) %>% as_tibble()
df_rules_fv <- DATAFRAME(rules_favorites,separate=TRUE) %>% as_tibble()

In [628]:
df_rules_pv$LHS <- df_rules_pv$LHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_rules_pv$RHS <- df_rules_pv$RHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_rules_fv$LHS <- df_rules_fv$LHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_rules_fv$RHS <- df_rules_fv$RHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_rules_pu$LHS <- df_rules_pu$LHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_rules_pu$RHS <- df_rules_pu$RHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_rules_ss_bysize$LHS <- df_rules_ss_bysize$LHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_rules_ss_bysize$RHS <- df_rules_ss_bysize$RHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_rules_ss_overall$LHS <- df_rules_ss_overall$LHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_rules_ss_overall$RHS <- df_rules_ss_overall$RHS %>% str_replace_all(c("[{]"="","[}]"=""))

In [41]:
inspect(df_rules_pu)

ERROR: Error in h(simpleError(msg, call)): error in evaluating the argument 'x' in selecting a method for function 'inspect': object 'df_rules_pu' not found


In [636]:
write_clip(#head(df_rules_ss_overall %>% mutate(KPI = 'samesize | purchases | see_top_overall_bundles', rank = row_number(desc(count))) %>% arrange(desc(count)),100) %>%
           #    union(head(df_rules_ss_bysize %>% filter(str_detect(LHS, '2.5x7')) %>% mutate(KPI = "samesize | purchases | see_top_bundles_by_size | 2.5x7", rank = row_number(desc(count))) %>% arrange(desc(count)),100)) %>%
           #    union(head(df_rules_ss_bysize %>% filter(str_detect(LHS, '2.5x10')) %>% mutate(KPI = "samesize | purchases | see_top_bundles_by_size | 2.5x10", rank = row_number(desc(count))) %>% arrange(desc(count)),100)) %>%
           #    union(head(df_rules_ss_bysize %>% filter(str_detect(LHS, '3x5')) %>% mutate(KPI = "samesize | purchases | see_top_bundles_by_size | 3x5", rank = row_number(desc(count))) %>% arrange(desc(count)),100)) %>%
           #    union(head(df_rules_ss_bysize %>% filter(str_detect(LHS, '5x7')) %>% mutate(KPI = "samesize | purchases | see_top_bundles_by_size | 5x7", rank = row_number(desc(count))) %>% arrange(desc(count)),100)) %>%
           #    union(head(df_rules_ss_bysize %>% filter(str_detect(LHS, '6x9')) %>% mutate(KPI = "samesize | purchases | see_top_bundles_by_size | 6x9", rank = row_number(desc(count))) %>% arrange(desc(count)),100)) %>%
           #    union(head(df_rules_ss_bysize %>% filter(str_detect(LHS, '8x10')) %>% mutate(KPI = "samesize | purchases | see_top_bundles_by_size | 8x10", rank = row_number(desc(count))) %>% arrange(desc(count)),100)) %>%
           #    union(head(df_rules_ss_bysize %>% filter(str_detect(LHS, '9x12')) %>% mutate(KPI = "samesize | purchases | see_top_bundles_by_size | 9x12", rank = row_number(desc(count))) %>% arrange(desc(count)),100)) %>%
           #    union(head(df_rules_ss_bysize %>% filter(str_detect(LHS, '6 round')) %>% mutate(KPI = "samesize | purchases | see_top_bundles_by_size | 6 Round", rank = row_number(desc(count))) %>% arrange(desc(count)),100)) %>%
           #    union(head(df_rules_ss_bysize %>% filter(str_detect(LHS, '8 round')) %>% mutate(KPI = "samesize | purchases | see_top_bundles_by_size | 8 Round", rank = row_number(desc(count))) %>% arrange(desc(count)),100)) %>%
           #    union(head(df_rules_ss_bysize %>% filter(str_detect(LHS, 'doormat')) %>% mutate(KPI = "samesize | purchases | create_bundle_by_type | multiple doormats", rank = row_number(desc(count))) %>% arrange(desc(count)),100)) %>%
           #union(head(df_rules_pu %>% mutate(KPI = "anysize | purchases | see_top_overall_bundles", rank = row_number(desc(count))) %>% arrange(desc(count)),100)) %>%
           #    union(head(df_rules_pu %>% filter(str_detect(LHS, 'plush')) %>% mutate(KPI = "anysize | purchases | create_bundle_by_type | plush", rank = row_number(desc(count))) %>% arrange(desc(count)),100)) %>%
           #    union(head(df_rules_pu %>% filter(str_detect(LHS, 'door')) %>% mutate(KPI = "anysize | purchases | create_bundle_by_type | doormat + outdoor", rank = row_number(desc(count))) %>% arrange(desc(count)),100))
           #head(df_rules_fv %>% mutate(KPI = "anysize | favorites | see_top_overall_bundles", rank = row_number(desc(count))) %>% arrange(desc(count)),100) %>%
           #    union(head(df_rules_fv %>% filter(str_detect(LHS, 'plush')) %>% mutate(KPI = "anysize | favorites | create_bundle_by_type | plush", rank = row_number(desc(count))) %>% arrange(desc(count)),100)) %>%
           #    union(head(df_rules_fv %>% filter(str_detect(LHS, 'door')) %>% mutate(KPI = "anysize | favorites | create_bundle_by_type | doormat + outdoor", rank = row_number(desc(count))) %>% arrange(desc(count)),100)) %>%
           #union(head(df_rules_pv %>% mutate(KPI = "anysize | pageviews | see_top_overall_bundles", rank = row_number(desc(count))) %>% arrange(desc(count)),100)) %>%
           #    union(head(df_rules_pv %>% filter(str_detect(LHS, 'plush')) %>% mutate(KPI = "anysize | pageviews | create_bundle_by_type | plush", rank = row_number(desc(count))) %>% arrange(desc(count)),100)) %>%
           #    union(head(df_rules_pv %>% filter(str_detect(LHS, 'door')) %>% mutate(KPI = "anysize | pageviews | create_bundle_by_type | doormat + outdoor", rank = row_number(desc(count))) %>% arrange(desc(count)),100))
           allow_non_interactive=TRUE)

### Rug-Specific Rules

In [ ]:
kamran coral rug
kamran hazel rug
verena dark wood rug
hendesi heriz abalone rug
damali black & white rug
impasto slate blue rug
moroccan diamond plush rug
camellia jade rug
kamran royal blue rug
delphina delft blue rug
desert sumac rug
celestine coral rug
cambria ruby rug
hudson stripe black rug
the rebellion rug
prisma monochrome rug
cambria abalone rug
floral medallion yellow grey rug
maral heriz creme rug
polar white plush rug
celestine teal blue rug
absida rainbow rug
thalia black rug
leelu black & white rug
terali slate rug
zareen scarlet red rug
rogue squadron dark teal rug
terali ash grey rug
watercolor floral coral rug
cambria sapphire rug
linear diamond black rug
hendesi heriz cream rug
impasto greyscale rug
sima sage rug
gradasi teal quartz rug
buffalo plaid black & white rug
vintage daisy bordered blue rug
serenata ash grey rug
delphina navy blue rug
delphina black & white rug
floral medallion multicolor rug
kavi mosaic silver rug
herdanza black & white rug
hendesi heriz sage rug
maral border vintage grey rug
suzani coral rug
linear striped multicolor rug
sima royal blue rug
dark side damask charcoal rug
verena sapphire rug


In [18]:
head(df_purchases)

customer_id,products_purchased
<dbl>,<chr>
7582895044,"3x5 - cynthia rowley suzani black rug, 5x7 - vintage daisy bordered tangerine rug"
403576619039,"8x10 - diamond dalia beige ivory rug - cover only, 8x10 - domo sage rug - cover only"
565915779103,"9x12 - kamran hazel rug, 2.5x10 - maral heriz creme rug"
577449328671,"8 round - delphina black & white rug, 5x7 - star wars armada black rug - cover only"
597340815452,"2.5x10 - cambria abalone rug, 2.5x10 - kamran hazel rug"
601774751836,"6x9 - floral medallion multicolor rug, 8x10 - jellyfish bloom silver rug, 8x10 - kavi diamond silver rug, 6x9 - moroccan diamond plush rug"


In [44]:
head(df_purchases %>% filter(grepl('hello', products_purchased))) 

customer_id,products_purchased
<dbl>,<chr>
7.215602e+11,"6x9 - damali black & white rug, 2x3 - hello doormat"
2.446251e+12,"2x3 - hello doormat, 6 round - impasto slate blue rug - cover only"
2.448179e+12,"3x5 - camellia jade rug, 2x3 - hello doormat, 2x3 - imperial monogram doormat"
2.582044e+12,"2x3 - hello doormat, 3x5 - lemonade yellow rug"
8.103567e+11,"2x3 - hello sunshine blue doormat, 2x3 - imperial monogram doormat, 8x10 - outdoor nomada multicolor rug, 2.5x10 - verena dark wood rug"
2.435185e+12,"2.5x10 - chloe trellis fuchsia pink rug, 5x7 - delphina black & white rug, 2x3 - hello doormat, 5x7 - witchcraft rug"


In [48]:
inspect(subset(select_rug_rules,lift > 1))

     lhs                      rhs                                                  support confidence    coverage      lift count
[1]  {2x3 - hello doormat} => {2x3 - alterna stripe doormat}                  6.130738e-05 0.01253918 0.004889264 20.452978     4
[2]  {2x3 - hello doormat} => {2x3 - henri botanique black doormat}           7.663422e-05 0.01567398 0.004889264 27.639160     5
[3]  {2x3 - hello doormat} => {2x3 - dog house grass doormat}                 6.130738e-05 0.01253918 0.004889264 24.062327     4
[4]  {2x3 - hello doormat} => {2x3 - looped solid dark coir doormat}          6.130738e-05 0.01253918 0.004889264 16.696309     4
[5]  {2x3 - hello doormat} => {3x5 - outdoor gingham plaid black & white rug} 7.663422e-05 0.01567398 0.004889264 17.631878     5
[6]  {2x3 - hello doormat} => {2x3 - adamas diamond doormat}                  7.663422e-05 0.01567398 0.004889264 17.044148     5
[7]  {2x3 - hello doormat} => {2x3 - looped solid light coir doormat}         6.130738e-05

In [674]:
damalibw_rules_ss_overall <- apriori(samesize_overall_purchase_transactions, parameter = list(supp=0.001, conf=0.01),appearance = list(lhs="damali black & white rug",default="rhs"))

Apriori

Parameter specification:
 confidence minval smax arem  aval originalSupport maxtime support minlen
       0.01    0.1    1 none FALSE            TRUE       5   0.001      1
 maxlen target  ext
     10  rules TRUE

Algorithmic control:
 filter tree heap memopt load sort verbose
    0.1 TRUE TRUE  FALSE TRUE    2    TRUE

Absolute minimum support count: 18 

set item appearances ...[1 item(s)] done [0.00s].
set transactions ...[1365 item(s), 18466 transaction(s)] done [0.01s].
sorting and recoding items ... [397 item(s)] done [0.00s].
creating transaction tree ... done [0.00s].
checking subsets of size 1 2 done [0.00s].
writing ... [39 rule(s)] done [0.00s].
creating S4 object  ... done [0.00s].


In [720]:
# based on 
kamrancoral_rules_ss_pu_overall <- apriori(samesize_overall_purchase_transactions, parameter = list(supp=0.0001, conf=0.01),appearance = list(lhs="kamran coral rug",default="rhs"))
kamranhazel_rules_ss_pu_overall <- apriori(samesize_overall_purchase_transactions, parameter = list(supp=0.0001, conf=0.01),appearance = list(lhs="kamran hazel rug",default="rhs"))
verenadarkwood_rules_ss_pu_overall <- apriori(samesize_overall_purchase_transactions, parameter = list(supp=0.0001, conf=0.01),appearance = list(lhs="verena dark wood rug",default="rhs"))
hendesiherizabalone_rules_ss_pu_overall <- apriori(samesize_overall_purchase_transactions, parameter = list(supp=0.0001, conf=0.01),appearance = list(lhs="hendesi heriz abalone rug",default="rhs"))
damaliblackandwhite_rules_ss_pu_overall <- apriori(samesize_overall_purchase_transactions, parameter = list(supp=0.0001, conf=0.01),appearance = list(lhs="damali black & white rug",default="rhs"))
impastoslateblue_rules_ss_pu_overall <- apriori(samesize_overall_purchase_transactions, parameter = list(supp=0.0001, conf=0.01),appearance = list(lhs="impasto slate blue rug",default="rhs"))
moroccandiamondplush_rules_ss_pu_overall <- apriori(samesize_overall_purchase_transactions, parameter = list(supp=0.0001, conf=0.01),appearance = list(lhs="moroccan diamond plush rug",default="rhs"))
camelliajade_rules_ss_pu_overall <- apriori(samesize_overall_purchase_transactions, parameter = list(supp=0.0001, conf=0.01),appearance = list(lhs="camellia jade rug",default="rhs"))
kamranroyalblue_rules_ss_pu_overall <- apriori(samesize_overall_purchase_transactions, parameter = list(supp=0.0001, conf=0.01),appearance = list(lhs="kamran royal blue rug",default="rhs"))
delphinadelftblue_rules_ss_pu_overall <- apriori(samesize_overall_purchase_transactions, parameter = list(supp=0.0001, conf=0.01),appearance = list(lhs="delphina delft blue rug",default="rhs"))
desertsumac_rules_ss_pu_overall <- apriori(samesize_overall_purchase_transactions, parameter = list(supp=0.0001, conf=0.01),appearance = list(lhs="desert sumac rug",default="rhs"))
celestinecoral_rules_ss_pu_overall <- apriori(samesize_overall_purchase_transactions, parameter = list(supp=0.0001, conf=0.01),appearance = list(lhs="celestine coral rug",default="rhs"))
cambriaruby_rules_ss_pu_overall <- apriori(samesize_overall_purchase_transactions, parameter = list(supp=0.0001, conf=0.01),appearance = list(lhs="cambria ruby rug",default="rhs"))
hudsonstripeblack_rules_ss_pu_overall <- apriori(samesize_overall_purchase_transactions, parameter = list(supp=0.0001, conf=0.01),appearance = list(lhs="hudson stripe black rug",default="rhs"))
therebellion_rules_ss_pu_overall <- apriori(samesize_overall_purchase_transactions, parameter = list(supp=0.0001, conf=0.01),appearance = list(lhs="the rebellion rug",default="rhs"))
prismamonochrome_rules_ss_pu_overall <- apriori(samesize_overall_purchase_transactions, parameter = list(supp=0.0001, conf=0.01),appearance = list(lhs="prisma monochrome rug",default="rhs"))
cambriaabalone_rules_ss_pu_overall <- apriori(samesize_overall_purchase_transactions, parameter = list(supp=0.0001, conf=0.01),appearance = list(lhs="cambria abalone rug",default="rhs"))
floralmedallionyellowgrey_rules_ss_pu_overall <- apriori(samesize_overall_purchase_transactions, parameter = list(supp=0.0001, conf=0.01),appearance = list(lhs="floral medallion yellow grey rug",default="rhs"))
maralherizcreme_rules_ss_pu_overall <- apriori(samesize_overall_purchase_transactions, parameter = list(supp=0.0001, conf=0.01),appearance = list(lhs="maral heriz creme rug",default="rhs"))
polarwhiteplush_rules_ss_pu_overall <- apriori(samesize_overall_purchase_transactions, parameter = list(supp=0.0001, conf=0.01),appearance = list(lhs="polar white plush rug",default="rhs"))
celestinetealblue_rules_ss_pu_overall <- apriori(samesize_overall_purchase_transactions, parameter = list(supp=0.0001, conf=0.01),appearance = list(lhs="celestine teal blue rug",default="rhs"))
absidarainbow_rules_ss_pu_overall <- apriori(samesize_overall_purchase_transactions, parameter = list(supp=0.0001, conf=0.01),appearance = list(lhs="absida rainbow rug",default="rhs"))
thaliablack_rules_ss_pu_overall <- apriori(samesize_overall_purchase_transactions, parameter = list(supp=0.0001, conf=0.01),appearance = list(lhs="thalia black rug",default="rhs"))
leelublackandwhite_rules_ss_pu_overall <- apriori(samesize_overall_purchase_transactions, parameter = list(supp=0.0001, conf=0.01),appearance = list(lhs="leelu black & white rug",default="rhs"))
teralislate_rules_ss_pu_overall <- apriori(samesize_overall_purchase_transactions, parameter = list(supp=0.0001, conf=0.01),appearance = list(lhs="terali slate rug",default="rhs"))
thaliablack_rules_ss_pu_overall <- apriori(samesize_overall_purchase_transactions, parameter = list(supp=0.0001, conf=0.01),appearance = list(lhs="thalia black rug",default="rhs"))

zareenscarletred_rules_ss_pu_overall <- apriori(samesize_overall_purchase_transactions, parameter = list(supp=0.0001, conf=0.01),appearance = list(lhs="zareen scarlet red rug",default="rhs"))
roguesquadrondarkteal_rules_ss_pu_overall <- apriori(samesize_overall_purchase_transactions, parameter = list(supp=0.0001, conf=0.01),appearance = list(lhs="rogue squadron dark teal rug",default="rhs"))
teraliashgrey_rules_ss_pu_overall <- apriori(samesize_overall_purchase_transactions, parameter = list(supp=0.0001, conf=0.01),appearance = list(lhs="terali ash grey rug",default="rhs"))
watercolorfloralcoral_rules_ss_pu_overall <- apriori(samesize_overall_purchase_transactions, parameter = list(supp=0.0001, conf=0.01),appearance = list(lhs="watercolor floral coral rug",default="rhs"))
cambriasapphire_rules_ss_pu_overall <- apriori(samesize_overall_purchase_transactions, parameter = list(supp=0.0001, conf=0.01),appearance = list(lhs="cambria sapphire rug",default="rhs"))
lineardiamondblack_rules_ss_pu_overall <- apriori(samesize_overall_purchase_transactions, parameter = list(supp=0.0001, conf=0.01),appearance = list(lhs="linear diamond black rug",default="rhs"))
hendesiherizcream_rules_ss_pu_overall <- apriori(samesize_overall_purchase_transactions, parameter = list(supp=0.0001, conf=0.01),appearance = list(lhs="hendesi heriz cream rug",default="rhs"))
impastogreyscale_rules_ss_pu_overall <- apriori(samesize_overall_purchase_transactions, parameter = list(supp=0.0001, conf=0.01),appearance = list(lhs="impasto greyscale rug",default="rhs"))
simasage_rules_ss_pu_overall <- apriori(samesize_overall_purchase_transactions, parameter = list(supp=0.0001, conf=0.01),appearance = list(lhs="sima sage rug",default="rhs"))
gradasitealquartz_rules_ss_pu_overall <- apriori(samesize_overall_purchase_transactions, parameter = list(supp=0.0001, conf=0.01),appearance = list(lhs="gradasi teal quartz rug",default="rhs"))
buffaloplaidblackandwhite_rules_ss_pu_overall <- apriori(samesize_overall_purchase_transactions, parameter = list(supp=0.0001, conf=0.01),appearance = list(lhs="buffalo plaid black & white rug",default="rhs"))
vintagedaisyborderedblue_rules_ss_pu_overall <- apriori(samesize_overall_purchase_transactions, parameter = list(supp=0.0001, conf=0.01),appearance = list(lhs="vintage daisy bordered blue rug",default="rhs"))
serenataashgrey_rules_ss_pu_overall <- apriori(samesize_overall_purchase_transactions, parameter = list(supp=0.0001, conf=0.01),appearance = list(lhs="serenata ash grey rug",default="rhs"))
delphinanavyblue_rules_ss_pu_overall <- apriori(samesize_overall_purchase_transactions, parameter = list(supp=0.0001, conf=0.01),appearance = list(lhs="delphina navy blue rug",default="rhs"))
delphinablackandwhite_rules_ss_pu_overall <- apriori(samesize_overall_purchase_transactions, parameter = list(supp=0.0001, conf=0.01),appearance = list(lhs="delphina black & white rug",default="rhs"))
floralmedallionmulticolor_rules_ss_pu_overall <- apriori(samesize_overall_purchase_transactions, parameter = list(supp=0.0001, conf=0.01),appearance = list(lhs="floral medallion multicolor rug",default="rhs"))
kavimosaicsilver_rules_ss_pu_overall <- apriori(samesize_overall_purchase_transactions, parameter = list(supp=0.0001, conf=0.01),appearance = list(lhs="kavi mosaic silver rug",default="rhs"))
herdanzablackandwhite_rules_ss_pu_overall <- apriori(samesize_overall_purchase_transactions, parameter = list(supp=0.0001, conf=0.01),appearance = list(lhs="herdanza black & white rug",default="rhs"))
hendesiherizsage_rules_ss_pu_overall <- apriori(samesize_overall_purchase_transactions, parameter = list(supp=0.0001, conf=0.01),appearance = list(lhs="hendesi heriz sage rug",default="rhs"))
#maralbordervintagegrey_rules_ss_pu_overall <- apriori(samesize_overall_purchase_transactions, parameter = list(supp=0.0001, conf=0.01),appearance = list(lhs="maral border vintage grey rug",default="rhs"))
suzanicoral_rules_ss_pu_overall <- apriori(samesize_overall_purchase_transactions, parameter = list(supp=0.0001, conf=0.01),appearance = list(lhs="suzani coral rug",default="rhs"))
linearstripedmulticolor_rules_ss_pu_overall <- apriori(samesize_overall_purchase_transactions, parameter = list(supp=0.0001, conf=0.01),appearance = list(lhs="linear striped multicolor rug",default="rhs"))
simaroyalblue_rules_ss_pu_overall <- apriori(samesize_overall_purchase_transactions, parameter = list(supp=0.0001, conf=0.01),appearance = list(lhs="sima royal blue rug",default="rhs"))
darksidedamaskcharcoal_rules_ss_pu_overall <- apriori(samesize_overall_purchase_transactions, parameter = list(supp=0.0001, conf=0.01),appearance = list(lhs="dark side damask charcoal rug",default="rhs"))
verenasapphire_rules_ss_pu_overall <- apriori(samesize_overall_purchase_transactions, parameter = list(supp=0.0001, conf=0.01),appearance = list(lhs="verena sapphire rug",default="rhs"))

df_kamrancoral_rules_ss_pu_overall                <- DATAFRAME(kamrancoral_rules_ss_pu_overall,separate=TRUE) %>% as_tibble()
df_kamranhazel_rules_ss_pu_overall           <- DATAFRAME(kamranhazel_rules_ss_pu_overall,separate=TRUE) %>% as_tibble()
df_verenadarkwood_rules_ss_pu_overall                <- DATAFRAME(verenadarkwood_rules_ss_pu_overall,separate=TRUE) %>% as_tibble()
df_hendesiherizabalone_rules_ss_pu_overall        <- DATAFRAME(hendesiherizabalone_rules_ss_pu_overall,separate=TRUE) %>% as_tibble()
df_damaliblackandwhite_rules_ss_pu_overall            <- DATAFRAME(damaliblackandwhite_rules_ss_pu_overall,separate=TRUE) %>% as_tibble()
df_impastoslateblue_rules_ss_pu_overall             <- DATAFRAME(impastoslateblue_rules_ss_pu_overall,separate=TRUE) %>% as_tibble()
df_moroccandiamondplush_rules_ss_pu_overall          <- DATAFRAME(moroccandiamondplush_rules_ss_pu_overall,separate=TRUE) %>% as_tibble()
df_camelliajade_rules_ss_pu_overall           <- DATAFRAME(camelliajade_rules_ss_pu_overall,separate=TRUE) %>% as_tibble()
df_kamranroyalblue_rules_ss_pu_overall          <- DATAFRAME(kamranroyalblue_rules_ss_pu_overall,separate=TRUE) %>% as_tibble()
df_delphinadelftblue_rules_ss_pu_overall           <- DATAFRAME(delphinadelftblue_rules_ss_pu_overall,separate=TRUE) %>% as_tibble()
df_desertsumac_rules_ss_pu_overall                <- DATAFRAME(desertsumac_rules_ss_pu_overall,separate=TRUE) %>% as_tibble()
df_celestinecoral_rules_ss_pu_overall             <- DATAFRAME(celestinecoral_rules_ss_pu_overall,separate=TRUE) %>% as_tibble()
df_cambriaruby_rules_ss_pu_overall             <- DATAFRAME(cambriaruby_rules_ss_pu_overall,separate=TRUE) %>% as_tibble()
df_hudsonstripeblack_rules_ss_pu_overall   <- DATAFRAME(hudsonstripeblack_rules_ss_pu_overall,separate=TRUE) %>% as_tibble()
df_therebellion_rules_ss_pu_overall               <- DATAFRAME(therebellion_rules_ss_pu_overall,separate=TRUE) %>% as_tibble()
df_prismamonochrome_rules_ss_pu_overall              <- DATAFRAME(prismamonochrome_rules_ss_pu_overall,separate=TRUE) %>% as_tibble()
df_cambriaabalone_rules_ss_pu_overall          <- DATAFRAME(cambriaabalone_rules_ss_pu_overall,separate=TRUE) %>% as_tibble()
df_floralmedallionyellowgrey_rules_ss_pu_overall                   <- DATAFRAME(floralmedallionyellowgrey_rules_ss_pu_overall,separate=TRUE) %>% as_tibble()
df_maralherizcreme_rules_ss_pu_overall           <- DATAFRAME(maralherizcreme_rules_ss_pu_overall,separate=TRUE) %>% as_tibble()
df_polarwhiteplush_rules_ss_pu_overall           <- DATAFRAME(polarwhiteplush_rules_ss_pu_overall,separate=TRUE) %>% as_tibble()
df_celestinetealblue_rules_ss_pu_overall           <- DATAFRAME(celestinetealblue_rules_ss_pu_overall,separate=TRUE) %>% as_tibble()
df_absidarainbow_rules_ss_pu_overall              <- DATAFRAME(absidarainbow_rules_ss_pu_overall,separate=TRUE) %>% as_tibble()
df_thaliablack_rules_ss_pu_overall            <- DATAFRAME(thaliablack_rules_ss_pu_overall,separate=TRUE) %>% as_tibble()
df_leelublackandwhite_rules_ss_pu_overall            <- DATAFRAME(leelublackandwhite_rules_ss_pu_overall,separate=TRUE) %>% as_tibble()
df_teralislate_rules_ss_pu_overall       <- DATAFRAME(teralislate_rules_ss_pu_overall,separate=TRUE) %>% as_tibble()

df_thaliablack_rules_ss_pu_overall       <- DATAFRAME(thaliablack_rules_ss_pu_overall,separate=TRUE) %>% as_tibble()

df_zareenscarletred_rules_ss_pu_overall          <- DATAFRAME(zareenscarletred_rules_ss_pu_overall,separate=TRUE) %>% as_tibble()
df_roguesquadrondarkteal_rules_ss_pu_overall      <- DATAFRAME(roguesquadrondarkteal_rules_ss_pu_overall,separate=TRUE) %>% as_tibble()
df_teraliashgrey_rules_ss_pu_overall               <- DATAFRAME(teraliashgrey_rules_ss_pu_overall,separate=TRUE) %>% as_tibble()
df_watercolorfloralcoral_rules_ss_pu_overall          <- DATAFRAME(watercolorfloralcoral_rules_ss_pu_overall,separate=TRUE) %>% as_tibble()
df_cambriasapphire_rules_ss_pu_overall  <- DATAFRAME(cambriasapphire_rules_ss_pu_overall,separate=TRUE) %>% as_tibble()
df_lineardiamondblack_rules_ss_pu_overall        <- DATAFRAME(lineardiamondblack_rules_ss_pu_overall,separate=TRUE) %>% as_tibble()
df_hendesiherizcream_rules_ss_pu_overall            <- DATAFRAME(hendesiherizcream_rules_ss_pu_overall,separate=TRUE) %>% as_tibble()
df_impastogreyscale_rules_ss_pu_overall         <- DATAFRAME(impastogreyscale_rules_ss_pu_overall,separate=TRUE) %>% as_tibble()
df_simasage_rules_ss_pu_overall  <- DATAFRAME(simasage_rules_ss_pu_overall,separate=TRUE) %>% as_tibble()
df_gradasitealquartz_rules_ss_pu_overall         <- DATAFRAME(gradasitealquartz_rules_ss_pu_overall,separate=TRUE) %>% as_tibble()
df_buffaloplaidblackandwhite_rules_ss_pu_overall                <- DATAFRAME(buffaloplaidblackandwhite_rules_ss_pu_overall,separate=TRUE) %>% as_tibble()
df_vintagedaisyborderedblue_rules_ss_pu_overall           <- DATAFRAME(vintagedaisyborderedblue_rules_ss_pu_overall,separate=TRUE) %>% as_tibble()
df_serenataashgrey_rules_ss_pu_overall          <- DATAFRAME(serenataashgrey_rules_ss_pu_overall,separate=TRUE) %>% as_tibble()
df_delphinanavyblue_rules_ss_pu_overall            <- DATAFRAME(delphinanavyblue_rules_ss_pu_overall,separate=TRUE) %>% as_tibble()
df_delphinablackandwhite_rules_ss_pu_overall    <- DATAFRAME(delphinablackandwhite_rules_ss_pu_overall,separate=TRUE) %>% as_tibble()
df_floralmedallionmulticolor_rules_ss_pu_overall                <- DATAFRAME(floralmedallionmulticolor_rules_ss_pu_overall,separate=TRUE) %>% as_tibble()
df_kavimosaicsilver_rules_ss_pu_overall   <- DATAFRAME(kavimosaicsilver_rules_ss_pu_overall,separate=TRUE) %>% as_tibble()
df_herdanzablackandwhite_rules_ss_pu_overall       <- DATAFRAME(herdanzablackandwhite_rules_ss_pu_overall,separate=TRUE) %>% as_tibble()
df_hendesiherizsage_rules_ss_pu_overall     <- DATAFRAME(hendesiherizsage_rules_ss_pu_overall,separate=TRUE) %>% as_tibble()
#df_maralbordervintagegrey_rules_ss_pu_overall <- DATAFRAME(maralbordervintagegrey_rules_ss_pu_overall,separate=TRUE) %>% as_tibble()
df_suzanicoral_rules_ss_pu_overall      <- DATAFRAME(suzanicoral_rules_ss_pu_overall,separate=TRUE) %>% as_tibble()
df_linearstripedmulticolor_rules_ss_pu_overall            <- DATAFRAME(linearstripedmulticolor_rules_ss_pu_overall,separate=TRUE) %>% as_tibble()
df_simaroyalblue_rules_ss_pu_overall   <- DATAFRAME(simaroyalblue_rules_ss_pu_overall,separate=TRUE) %>% as_tibble()
df_darksidedamaskcharcoal_rules_ss_pu_overall                <- DATAFRAME(darksidedamaskcharcoal_rules_ss_pu_overall,separate=TRUE) %>% as_tibble()
df_verenasapphire_rules_ss_pu_overall       <- DATAFRAME(verenasapphire_rules_ss_pu_overall,separate=TRUE) %>% as_tibble()
df_thaliablack_rules_ss_pu_overall       <- DATAFRAME(thaliablack_rules_ss_pu_overall,separate=TRUE) %>% as_tibble()

df_kamrancoral_rules_ss_pu_overall$LHS                <- df_kamrancoral_rules_ss_pu_overall$LHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_kamranhazel_rules_ss_pu_overall$LHS           <- df_kamranhazel_rules_ss_pu_overall$LHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_verenadarkwood_rules_ss_pu_overall$LHS                <- df_verenadarkwood_rules_ss_pu_overall$LHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_hendesiherizabalone_rules_ss_pu_overall$LHS        <- df_hendesiherizabalone_rules_ss_pu_overall$LHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_damaliblackandwhite_rules_ss_pu_overall$LHS            <- df_damaliblackandwhite_rules_ss_pu_overall$LHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_impastoslateblue_rules_ss_pu_overall$LHS             <- df_impastoslateblue_rules_ss_pu_overall$LHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_moroccandiamondplush_rules_ss_pu_overall$LHS          <- df_moroccandiamondplush_rules_ss_pu_overall$LHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_camelliajade_rules_ss_pu_overall$LHS           <- df_camelliajade_rules_ss_pu_overall$LHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_kamranroyalblue_rules_ss_pu_overall$LHS          <- df_kamranroyalblue_rules_ss_pu_overall$LHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_delphinadelftblue_rules_ss_pu_overall$LHS           <- df_delphinadelftblue_rules_ss_pu_overall$LHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_desertsumac_rules_ss_pu_overall$LHS                <- df_desertsumac_rules_ss_pu_overall$LHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_celestinecoral_rules_ss_pu_overall$LHS             <- df_celestinecoral_rules_ss_pu_overall$LHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_cambriaruby_rules_ss_pu_overall$LHS             <- df_cambriaruby_rules_ss_pu_overall$LHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_hudsonstripeblack_rules_ss_pu_overall$LHS   <- df_hudsonstripeblack_rules_ss_pu_overall$LHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_therebellion_rules_ss_pu_overall$LHS               <- df_therebellion_rules_ss_pu_overall$LHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_prismamonochrome_rules_ss_pu_overall$LHS              <- df_prismamonochrome_rules_ss_pu_overall$LHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_cambriaabalone_rules_ss_pu_overall$LHS          <- df_cambriaabalone_rules_ss_pu_overall$LHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_floralmedallionyellowgrey_rules_ss_pu_overall$LHS                   <- df_floralmedallionyellowgrey_rules_ss_pu_overall$LHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_maralherizcreme_rules_ss_pu_overall$LHS           <- df_maralherizcreme_rules_ss_pu_overall$LHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_polarwhiteplush_rules_ss_pu_overall$LHS           <- df_polarwhiteplush_rules_ss_pu_overall$LHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_celestinetealblue_rules_ss_pu_overall$LHS           <- df_celestinetealblue_rules_ss_pu_overall$LHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_absidarainbow_rules_ss_pu_overall$LHS              <- df_absidarainbow_rules_ss_pu_overall$LHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_thaliablack_rules_ss_pu_overall$LHS            <- df_thaliablack_rules_ss_pu_overall$LHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_leelublackandwhite_rules_ss_pu_overall$LHS            <- df_leelublackandwhite_rules_ss_pu_overall$LHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_teralislate_rules_ss_pu_overall$LHS       <- df_teralislate_rules_ss_pu_overall$LHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_zareenscarletred_rules_ss_pu_overall$LHS          <- df_zareenscarletred_rules_ss_pu_overall$LHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_roguesquadrondarkteal_rules_ss_pu_overall$LHS      <- df_roguesquadrondarkteal_rules_ss_pu_overall$LHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_teraliashgrey_rules_ss_pu_overall$LHS               <- df_teraliashgrey_rules_ss_pu_overall$LHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_watercolorfloralcoral_rules_ss_pu_overall$LHS          <- df_watercolorfloralcoral_rules_ss_pu_overall$LHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_cambriasapphire_rules_ss_pu_overall$LHS  <- df_cambriasapphire_rules_ss_pu_overall$LHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_lineardiamondblack_rules_ss_pu_overall$LHS        <- df_lineardiamondblack_rules_ss_pu_overall$LHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_hendesiherizcream_rules_ss_pu_overall$LHS            <- df_hendesiherizcream_rules_ss_pu_overall$LHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_impastogreyscale_rules_ss_pu_overall$LHS         <- df_impastogreyscale_rules_ss_pu_overall$LHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_simasage_rules_ss_pu_overall$LHS  <- df_simasage_rules_ss_pu_overall$LHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_gradasitealquartz_rules_ss_pu_overall$LHS         <- df_gradasitealquartz_rules_ss_pu_overall$LHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_buffaloplaidblackandwhite_rules_ss_pu_overall$LHS                <- df_buffaloplaidblackandwhite_rules_ss_pu_overall$LHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_vintagedaisyborderedblue_rules_ss_pu_overall$LHS           <- df_vintagedaisyborderedblue_rules_ss_pu_overall$LHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_serenataashgrey_rules_ss_pu_overall$LHS          <- df_serenataashgrey_rules_ss_pu_overall$LHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_delphinanavyblue_rules_ss_pu_overall$LHS            <- df_delphinanavyblue_rules_ss_pu_overall$LHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_delphinablackandwhite_rules_ss_pu_overall$LHS    <- df_delphinablackandwhite_rules_ss_pu_overall$LHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_floralmedallionmulticolor_rules_ss_pu_overall$LHS                <- df_floralmedallionmulticolor_rules_ss_pu_overall$LHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_kavimosaicsilver_rules_ss_pu_overall$LHS   <- df_kavimosaicsilver_rules_ss_pu_overall$LHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_herdanzablackandwhite_rules_ss_pu_overall$LHS       <- df_herdanzablackandwhite_rules_ss_pu_overall$LHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_hendesiherizsage_rules_ss_pu_overall$LHS     <- df_hendesiherizsage_rules_ss_pu_overall$LHS %>% str_replace_all(c("[{]"="","[}]"=""))
#df_maralbordervintagegrey_rules_ss_pu_overall$LHS <- df_watercolorherringbonecream_rules_fv$df_maralbordervintagegrey_rules_ss_pu_overall %>% str_replace_all(c("[{]"="","[}]"=""))
df_suzanicoral_rules_ss_pu_overall$LHS      <- df_suzanicoral_rules_ss_pu_overall$LHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_linearstripedmulticolor_rules_ss_pu_overall$LHS            <- df_linearstripedmulticolor_rules_ss_pu_overall$LHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_simaroyalblue_rules_ss_pu_overall$LHS   <- df_simaroyalblue_rules_ss_pu_overall$LHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_darksidedamaskcharcoal_rules_ss_pu_overall$LHS                <- df_darksidedamaskcharcoal_rules_ss_pu_overall$LHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_verenasapphire_rules_ss_pu_overall$LHS       <- df_verenasapphire_rules_ss_pu_overall$LHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_thaliablack_rules_ss_pu_overall$LHS       <- df_thaliablack_rules_ss_pu_overall$LHS %>% str_replace_all(c("[{]"="","[}]"=""))

df_kamrancoral_rules_ss_pu_overall$RHS                <- df_kamrancoral_rules_ss_pu_overall$RHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_kamranhazel_rules_ss_pu_overall$RHS           <- df_kamranhazel_rules_ss_pu_overall$RHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_verenadarkwood_rules_ss_pu_overall$RHS                <- df_verenadarkwood_rules_ss_pu_overall$RHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_hendesiherizabalone_rules_ss_pu_overall$RHS        <- df_hendesiherizabalone_rules_ss_pu_overall$RHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_damaliblackandwhite_rules_ss_pu_overall$RHS            <- df_damaliblackandwhite_rules_ss_pu_overall$RHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_impastoslateblue_rules_ss_pu_overall$RHS             <- df_impastoslateblue_rules_ss_pu_overall$RHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_moroccandiamondplush_rules_ss_pu_overall$RHS          <- df_moroccandiamondplush_rules_ss_pu_overall$RHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_camelliajade_rules_ss_pu_overall$RHS           <- df_camelliajade_rules_ss_pu_overall$RHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_kamranroyalblue_rules_ss_pu_overall$RHS          <- df_kamranroyalblue_rules_ss_pu_overall$RHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_delphinadelftblue_rules_ss_pu_overall$RHS           <- df_delphinadelftblue_rules_ss_pu_overall$RHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_desertsumac_rules_ss_pu_overall$RHS                <- df_desertsumac_rules_ss_pu_overall$RHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_celestinecoral_rules_ss_pu_overall$RHS             <- df_celestinecoral_rules_ss_pu_overall$RHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_cambriaruby_rules_ss_pu_overall$RHS             <- df_cambriaruby_rules_ss_pu_overall$RHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_hudsonstripeblack_rules_ss_pu_overall$RHS   <- df_hudsonstripeblack_rules_ss_pu_overall$RHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_therebellion_rules_ss_pu_overall$RHS               <- df_therebellion_rules_ss_pu_overall$RHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_prismamonochrome_rules_ss_pu_overall$RHS              <- df_prismamonochrome_rules_ss_pu_overall$RHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_cambriaabalone_rules_ss_pu_overall$RHS          <- df_cambriaabalone_rules_ss_pu_overall$RHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_floralmedallionyellowgrey_rules_ss_pu_overall$RHS                   <- df_floralmedallionyellowgrey_rules_ss_pu_overall$RHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_maralherizcreme_rules_ss_pu_overall$RHS           <- df_maralherizcreme_rules_ss_pu_overall$RHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_polarwhiteplush_rules_ss_pu_overall$RHS           <- df_polarwhiteplush_rules_ss_pu_overall$RHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_celestinetealblue_rules_ss_pu_overall$RHS           <- df_celestinetealblue_rules_ss_pu_overall$RHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_absidarainbow_rules_ss_pu_overall$RHS              <- df_absidarainbow_rules_ss_pu_overall$RHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_thaliablack_rules_ss_pu_overall$RHS            <- df_thaliablack_rules_ss_pu_overall$RHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_leelublackandwhite_rules_ss_pu_overall$RHS            <- df_leelublackandwhite_rules_ss_pu_overall$RHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_teralislate_rules_ss_pu_overall$RHS       <- df_teralislate_rules_ss_pu_overall$RHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_zareenscarletred_rules_ss_pu_overall$RHS          <- df_zareenscarletred_rules_ss_pu_overall$RHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_roguesquadrondarkteal_rules_ss_pu_overall$RHS      <- df_roguesquadrondarkteal_rules_ss_pu_overall$RHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_teraliashgrey_rules_ss_pu_overall$RHS               <- df_teraliashgrey_rules_ss_pu_overall$RHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_watercolorfloralcoral_rules_ss_pu_overall$RHS          <- df_watercolorfloralcoral_rules_ss_pu_overall$RHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_cambriasapphire_rules_ss_pu_overall$RHS  <- df_cambriasapphire_rules_ss_pu_overall$RHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_lineardiamondblack_rules_ss_pu_overall$RHS        <- df_lineardiamondblack_rules_ss_pu_overall$RHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_hendesiherizcream_rules_ss_pu_overall$RHS            <- df_hendesiherizcream_rules_ss_pu_overall$RHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_impastogreyscale_rules_ss_pu_overall$RHS         <- df_impastogreyscale_rules_ss_pu_overall$RHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_simasage_rules_ss_pu_overall$RHS  <- df_simasage_rules_ss_pu_overall$RHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_gradasitealquartz_rules_ss_pu_overall$RHS         <- df_gradasitealquartz_rules_ss_pu_overall$RHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_buffaloplaidblackandwhite_rules_ss_pu_overall$RHS                <- df_buffaloplaidblackandwhite_rules_ss_pu_overall$RHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_vintagedaisyborderedblue_rules_ss_pu_overall$RHS           <- df_vintagedaisyborderedblue_rules_ss_pu_overall$RHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_serenataashgrey_rules_ss_pu_overall$RHS          <- df_serenataashgrey_rules_ss_pu_overall$RHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_delphinanavyblue_rules_ss_pu_overall$RHS            <- df_delphinanavyblue_rules_ss_pu_overall$RHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_delphinablackandwhite_rules_ss_pu_overall$RHS    <- df_delphinablackandwhite_rules_ss_pu_overall$RHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_floralmedallionmulticolor_rules_ss_pu_overall$RHS                <- df_floralmedallionmulticolor_rules_ss_pu_overall$RHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_kavimosaicsilver_rules_ss_pu_overall$RHS   <- df_kavimosaicsilver_rules_ss_pu_overall$RHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_herdanzablackandwhite_rules_ss_pu_overall$RHS       <- df_herdanzablackandwhite_rules_ss_pu_overall$RHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_hendesiherizsage_rules_ss_pu_overall$RHS     <- df_hendesiherizsage_rules_ss_pu_overall$RHS %>% str_replace_all(c("[{]"="","[}]"=""))
#df_maralbordervintagegrey_rules_ss_pu_overall$RHS <- df_watercolorherringbonecream_rules_fv$df_maralbordervintagegrey_rules_ss_pu_overall %>% str_replace_all(c("[{]"="","[}]"=""))
df_suzanicoral_rules_ss_pu_overall$RHS      <- df_suzanicoral_rules_ss_pu_overall$RHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_linearstripedmulticolor_rules_ss_pu_overall$RHS            <- df_linearstripedmulticolor_rules_ss_pu_overall$RHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_simaroyalblue_rules_ss_pu_overall$RHS   <- df_simaroyalblue_rules_ss_pu_overall$RHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_darksidedamaskcharcoal_rules_ss_pu_overall$RHS                <- df_darksidedamaskcharcoal_rules_ss_pu_overall$RHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_verenasapphire_rules_ss_pu_overall$RHS       <- df_verenasapphire_rules_ss_pu_overall$RHS %>% str_replace_all(c("[{]"="","[}]"=""))
df_thaliablack_rules_ss_pu_overall$RHS       <- df_thaliablack_rules_ss_pu_overall$RHS %>% str_replace_all(c("[{]"="","[}]"=""))


Apriori

Parameter specification:
 confidence minval smax arem  aval originalSupport maxtime support minlen
       0.01    0.1    1 none FALSE            TRUE       5   1e-04      1
 maxlen target  ext
     10  rules TRUE

Algorithmic control:
 filter tree heap memopt load sort verbose
    0.1 TRUE TRUE  FALSE TRUE    2    TRUE

Absolute minimum support count: 1 

set item appearances ...[1 item(s)] done [0.00s].
set transactions ...[1365 item(s), 18466 transaction(s)] done [0.01s].
sorting and recoding items ... [1128 item(s)] done [0.00s].
creating transaction tree ... done [0.00s].
checking subsets of size 1 2 done [0.00s].
writing ... [49 rule(s)] done [0.00s].
creating S4 object  ... done [0.00s].
Apriori

Parameter specification:
 confidence minval smax arem  aval originalSupport maxtime support minlen
       0.01    0.1    1 none FALSE            TRUE       5   1e-04      1
 maxlen target  ext
     10  rules TRUE

Algorithmic control:
 filter tree heap memopt load sort verbose
 

In [716]:
write_clip(head(df_kamrancoral_rules_ss_pu_overall %>% filter(lift > 1.5) %>% mutate(KPI = 'samesize | purchases | create_bundle_by_design | kamrancoral', rank = row_number(desc(count))) %>% arrange(desc(count)),10) %>%
               union(head(df_kamranhazel_rules_ss_pu_overall %>% filter(lift > 1.5) %>% mutate(KPI = 'samesize | purchases | create_bundle_by_design | kamranhazel', rank = row_number(desc(count))) %>% arrange(desc(count)),10)) %>%
               union(head(df_verenadarkwood_rules_ss_pu_overall %>% filter(lift > 1.5) %>% mutate(KPI = 'samesize | purchases | create_bundle_by_design | verenadarkwood', rank = row_number(desc(count))) %>% arrange(desc(count)),10)) %>%
               union(head(df_hendesiherizabalone_rules_ss_pu_overall %>% filter(lift > 1.5) %>% mutate(KPI = 'samesize | purchases | create_bundle_by_design | hendesiherizabalone', rank = row_number(desc(count))) %>% arrange(desc(count)),10)) %>%
               union(head(df_damaliblackandwhite_rules_ss_pu_overall %>% filter(lift > 1.5) %>% mutate(KPI = 'samesize | purchases | create_bundle_by_design | damaliblackandwhite', rank = row_number(desc(count))) %>% arrange(desc(count)),10)) %>%
               union(head(df_impastoslateblue_rules_ss_pu_overall %>% filter(lift > 1.5) %>% mutate(KPI = 'samesize | purchases | create_bundle_by_design | impastoslateblue', rank = row_number(desc(count))) %>% arrange(desc(count)),10)) %>%
               union(head(df_moroccandiamondplush_rules_ss_pu_overall %>% filter(lift > 1.5) %>% mutate(KPI = 'samesize | purchases | create_bundle_by_design | moroccandiamondplush', rank = row_number(desc(count))) %>% arrange(desc(count)),10)) %>%
               union(head(df_camelliajade_rules_ss_pu_overall %>% filter(lift > 1.5) %>% mutate(KPI = 'samesize | purchases | create_bundle_by_design | camelliajade', rank = row_number(desc(count))) %>% arrange(desc(count)),10)) %>%
               union(head(df_kamranroyalblue_rules_ss_pu_overall %>% filter(lift > 1.5) %>% mutate(KPI = 'samesize | purchases | create_bundle_by_design | kamranroyalblue', rank = row_number(desc(count))) %>% arrange(desc(count)),10)) %>%
               union(head(df_delphinadelftblue_rules_ss_pu_overall %>% filter(lift > 1.5) %>% mutate(KPI = 'samesize | purchases | create_bundle_by_design | delphinadelftblue', rank = row_number(desc(count))) %>% arrange(desc(count)),10)) %>%
               union(head(df_desertsumac_rules_ss_pu_overall %>% filter(lift > 1.5) %>% mutate(KPI = 'samesize | purchases | create_bundle_by_design | desertsumac', rank = row_number(desc(count))) %>% arrange(desc(count)),10)) %>%
               union(head(df_celestinecoral_rules_ss_pu_overall %>% filter(lift > 1.5) %>% mutate(KPI = 'samesize | purchases | create_bundle_by_design | celestinecoral', rank = row_number(desc(count))) %>% arrange(desc(count)),10)) %>%
               union(head(df_cambriaruby_rules_ss_pu_overall %>% filter(lift > 1.5) %>% mutate(KPI = 'samesize | purchases | create_bundle_by_design | cambriaruby', rank = row_number(desc(count))) %>% arrange(desc(count)),10)) %>%
               union(head(df_hudsonstripeblack_rules_ss_pu_overall %>% filter(lift > 1.5) %>% mutate(KPI = 'samesize | purchases | create_bundle_by_design | hudsonstripeblack', rank = row_number(desc(count))) %>% arrange(desc(count)),10)) %>%
               union(head(df_therebellion_rules_ss_pu_overall %>% filter(lift > 1.5) %>% mutate(KPI = 'samesize | purchases | create_bundle_by_design | therebellion', rank = row_number(desc(count))) %>% arrange(desc(count)),10)) %>%
               union(head(df_prismamonochrome_rules_ss_pu_overall %>% filter(lift > 1.5) %>% mutate(KPI = 'samesize | purchases | create_bundle_by_design | prismamonochrome', rank = row_number(desc(count))) %>% arrange(desc(count)),10)) %>%
               union(head(df_cambriaabalone_rules_ss_pu_overall %>% filter(lift > 1.5) %>% mutate(KPI = 'samesize | purchases | create_bundle_by_design | cambriaabalone', rank = row_number(desc(count))) %>% arrange(desc(count)),10)) %>%
               union(head(df_floralmedallionyellowgrey_rules_ss_pu_overall %>% filter(lift > 1.5) %>% mutate(KPI = 'samesize | purchases | create_bundle_by_design | floralmedallionyellowgrey', rank = row_number(desc(count))) %>% arrange(desc(count)),10)) %>%
               union(head(df_maralherizcreme_rules_ss_pu_overall %>% filter(lift > 1.5) %>% mutate(KPI = 'samesize | purchases | create_bundle_by_design | maralherizcreme', rank = row_number(desc(count))) %>% arrange(desc(count)),10)) %>%
               union(head(df_polarwhiteplush_rules_ss_pu_overall %>% filter(lift > 1.5) %>% mutate(KPI = 'samesize | purchases | create_bundle_by_design | polarwhiteplush', rank = row_number(desc(count))) %>% arrange(desc(count)),10)) %>%
               union(head(df_celestinetealblue_rules_ss_pu_overall %>% filter(lift > 1.5) %>% mutate(KPI = 'samesize | purchases | create_bundle_by_design | celestinetealblue', rank = row_number(desc(count))) %>% arrange(desc(count)),10)) %>%
               union(head(df_absidarainbow_rules_ss_pu_overall %>% filter(lift > 1.5) %>% mutate(KPI = 'samesize | purchases | create_bundle_by_design | absidarainbow', rank = row_number(desc(count))) %>% arrange(desc(count)),10)) %>%
               union(head(df_thaliablack_rules_ss_pu_overall %>% filter(lift > 1.5) %>% mutate(KPI = 'samesize | purchases | create_bundle_by_design | thaliablack', rank = row_number(desc(count))) %>% arrange(desc(count)),10)) %>%
               union(head(df_leelublackandwhite_rules_ss_pu_overall %>% filter(lift > 1.5) %>% mutate(KPI = 'samesize | purchases | create_bundle_by_design | leelublackandwhite', rank = row_number(desc(count))) %>% arrange(desc(count)),10)) %>%
               union(head(df_teralislate_rules_ss_pu_overall %>% filter(lift > 1.5) %>% mutate(KPI = 'samesize | purchases | create_bundle_by_design | teralislate', rank = row_number(desc(count))) %>% arrange(desc(count)),10))

           ,allow_non_interactive=TRUE)


In [721]:
write_clip(head(df_thaliablack_rules_ss_pu_overall %>% filter(lift > 1.5) %>% mutate(KPI = 'samesize | purchases | create_bundle_by_design | thaliablack', rank = row_number(desc(count))) %>% arrange(desc(count)),10),allow_non_interactive=TRUE)

In [655]:
head(df_kamrancoral_rules_fv %>% filter(lift > 1.5) %>% mutate(KPI = 'anysize | favorites | create_bundle_by_design', rank = row_number()))

ERROR: Error in h(simpleError(msg, call)): error in evaluating the argument 'x' in selecting a method for function 'head': object 'df_kamrancoral_rules_fv' not found


In [ ]:
kamran-coral
impasto-slate-blue
kamran-hazel
hendesi-heriz-abalone
kamran-royal-blue
verena-dark-wood
celestine-teal-blue
prisma-monochrome
delphina-delft-blue
damali-black-white
terali-slate
cambria-abalone
celestine-coral
vintage-daisy-bordered-blue
camellia-jade
terali-ash-grey
serenata-slate-blue
sima-sage
kavi-mosaic-silver
impasto-greyscale
delphina-navy-blue
sima-royal-blue
maral-heriz-creme
leelu-black-white
sudaria-solid-teal-blue
celestine-sapphire
granite-ombre-light-grey
dakotah-sumac
hendesi-heriz-cream
floral-medallion-multicolor
malileh-heriz-saffron
prisma-slate-blue
linear-striped-multi
floral-medallion-yellow-grey
herdanza-black-white
thalia-black
hendesi-heriz-sage
gradasi-teal-quartz
serenata-ash-grey
herringbone-batik-ash-grey
cambria-ruby
watercolor-waves-sandstone
soraya-trellis-ash-grey
maral-border-vintage-grey
watercolor-herringbone-cream
watercolor-floral-coral
cambria-sapphire
vintage-daisy-bordered-grey
sima-abalone
vintage-heriz-cerulean
